# 案例要求


根据 “人员名片信息.xlsx”中所有的人员，按照“名片模板.docx”文件格式生成的名片

<table width="100%" border="0">
<tr>
  <td valign="top" style="vertical-align:top; text-align:left;background-color:#ddffdd;">
 
 **<font color="blue" size="3px"> 名片模板.docx（Word）</font>**
 
<img src="images/名片模板.png" align="center" style="width:40%"/>
  </td>
  </tr>
    <tr>
  <td style="vertical-align:top; text-align:left;background-color:#ddffdd">

**<font color="blue" size="3px"> 人员名片信息.xlsx（Excel）</font>**
 
<img src="images/人员名片信息.png" align="center"/>

  </td>
 </tr>
</table>

In [ ]:
# 导入工具包
import docx
from docx import Document
import copy # 复制模板用
import xlrd

In [ ]:
# 为了防止课程产生的文件过多，不方便文件的管理
# 本课程相关的文件都放到 files_dir 指定的目录下
files_dir = "02_files"

# 读取“人员名片信息.xlsx”函数

In [ ]:
def get_namecard_data(xls):
    """
    定义函数：从Excel文件中读取名片数据
    :param xls: Excel 文件
    :return: 返回了 2 组数据
             1）名片字段列表：['姓名', '职位', '部门', '电话', '手机', '邮箱']
             2）名片信息列表（列表中的列表）
             [['张三', '开发工程师', '技术部'..],['李四', '业务经理', ...]]
    """
    # 打开文件
    workbook = xlrd.open_workbook(xls)
    # 获取第一个工作表，从0开始
    sheet = workbook.sheet_by_index(0)
    # 名片信息列表（列表中的列表）
    namecard_infos_list = []
    # 从 0 开始
    field_list = sheet.row_values(0)
    for row_index in range(1, sheet.nrows):
        namecard_infos_list.append(sheet.row_values(row_index))
    # 返回了 2 组数据
    #   1）名片字段列表：['姓名', '职位', '部门', '电话', '手机', '邮箱']
    #   2）名片信息列表（列表中的列表）
    #      [['张三', '开发工程师', '技术部'..],['李四', '业务经理', ...]]
    return field_list, namecard_infos_list

In [ ]:
staff_namecard_info_file = f"./{files_dir}/人员名片信息.xlsx"

# 获取的人员名片信息有2组返回值，这里我们只赋值给一个变量
namecards = get_namecard_data(staff_namecard_info_file)
print(namecards) # 格式为：( [ ....],[....] ) 最外面是圆括号，是因为返回了2个参数

In [ ]:
# 获取的人员名片信息有2组返回值，我们建议赋值给2个变量，主要是方便操作
namecard_field_list, namecard_infos_list = get_namecard_data(staff_namecard_info_file)
print("名片字段：",namecard_field_list)
print("人员名片信息：",namecard_infos_list)

In [ ]:
from docx_ext import read_docx
# 查看一下模版中的数据
docx_template_file = f"./{files_dir}/Word_名片模版.docx"
read_docx(docx_template_file)

# 为了方便代码的阅读，建立几个函数

## 字典语法

In [ ]:
# 定义一个空字典
trans = {}  #  或者 trans = dict()
print(trans)

# 中英文翻译
trans["hello"] = "你好"
trans["bye"] = "再见"

print(trans)

## 新增函数：将替换列表改为替换字典

In [ ]:
def gen_replace_dict(namecard_infos):
    """
    生成替换字典
    """
    replace_dict = {} # 创建一个空字典
    for idx, namecard_info in enumerate(namecard_infos):
        field = namecard_field_list[idx]
        param = "[%s]" % field
        replace_dict[param] = namecard_info
    return replace_dict

## 测试函数（gen_replace_dict）

In [ ]:
namecards = ['刘德泽', '开发工程师', '技术部', '87646000', '13399999999', 'ldz@abcde.com.cn']
replace_dict = gen_replace_dict(namecards)
print("字段信息：",namecard_field_list)
print("名片信息：",namecards)
print("字典信息：",replace_dict)

## 新增函数：替换文字块内容

In [ ]:
def replace_run(run, replace_dict):
    """
    用于替换 run 对象中的值，比如：[姓名] => 张三
    (**注意** 在 replace_doc 函数中 创建的函数，只能在 replace_doc函数 中使用)
    :param run: 文字块
    :return:
    """
    for key in replace_dict.keys():
        # print(key, replace_dict[key])
        if key in run.text:
            run.text = run.text.replace(key, replace_dict[key])

## 测试函数（replace_run）

In [ ]:
doc = Document()
print("替换字典 =>",replace_dict)
p1 = doc.add_paragraph('姓名：[姓名]')
print("段落替换前 =>",p1.text)
replace_run(p1.runs[0],replace_dict)
print("段落替换后 =>",p1.text)

## 新增函数：遍历替换文档的段落

In [ ]:
def replace_doc(document, replace_dict):
    """
    替换文档中的 [姓名] [职位] 为 指定的名片信息
    :param document: 包含 [姓名] [职位]...的文档模板对象
    :param replace_dict: 
    :return:
    """
    # 循环遍历段落
    for paragraph in document.paragraphs:
        for run in paragraph.runs:
            replace_run(run, replace_dict)

    return document

## 函数准备好了，开始名片批量生成

In [ ]:
# 获取文档对象（作为名片模版）
name_card_tpl = docx.Document(docx_template_file)

# 循环遍历名片信息列表（多个人员的名片数据）
for namecard_info in namecard_infos_list:
    # 每次都要复制一份模板，防止原始的模板被替换了
    name_card_doc = copy.deepcopy(name_card_tpl)
    # 开始替换文档内容（参数分别为：docx模板副本 和 当前员工的名片数据）
    # namecard_info=['刘德泽', '开发工程师', '技术部', '87646000', '13399999999', 'ldz@abcde.com.cn']
#     replace_doc(name_card_doc, namecard_info)
    replace_dict = gen_replace_dict(namecard_info)
    replace_doc(name_card_doc,replace_dict )
    # 获取人员姓名
    staff_name = namecard_info[0]
    name_card_doc.save(f'./{files_dir}/{staff_name}的名片.docx')

# Python总结

1、字典的用法

2、创建函数可以提高代码的阅读性

# 思考题

万一，模板中一个段落中的内容把[姓名]放在两个文字块中，怎么办？

In [ ]:
# 比如这个模板代码

这个思考题有点难度，能解决的朋友，已经非常厉害了，不能解决，思考也不是坏事，继续努力。